In [2]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

# pre process data module
from ProcessCalcium import ProcessCalcium

# 
data_dir = '/media/cat/1TB/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'



Autosaving every 180 seconds


In [5]:
###########################################################################
##### PROCESS [CA] DATA (EVENT TRIGGERED MAPS): ROI OR PCA TIME SERIES ####
###########################################################################
'''  Compute calcium activity in ROIs selected (35) for 
     lever pull actiivty
     
     or just use PCA time courses 
'''

# Processing flags
parallel = False  # THIS DOESN"T REALLY WORK TOO MUCH MEMORY...
n_cores = 3

# select animal names
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datase
# names = ['IA1']


# window to compute
n_sec_window = 15
lockout_window = 10   # no of seconds of previous movement / lever pull lockout
recompute = True      # overwrite previously generated data

# # save PCA vs. ROI data; if selected, each dataset will be reduced to the PCs rquired to get to the explained var value
pca_etm = True
pca_explained_var_val = 0.95

################################
etm = ProcessCalcium()
etm.main_dir = data_dir
etm.verbose = True

#
etm.random_events_lockout = 3.0  # minimum number of seconds difference between a rewarded pull and a random/control 
etm.n_sec_window = n_sec_window 
etm.recompute = recompute
etm.pca_fixed_comps = 30  # fixed numer of components overrides explained_variance amount above

# 
# etm.remove_shift = True     # don't use shift
#etm.pca_n_components = 20
etm.whole_stack = True
etm.feature_name = "whole_stack"

#
#features = ['left_paw','right_paw','jaw']

# 
etm.feature_quiescence = 3    # number of seconds of no-movement prior to feature movement ;not applicable to code-04
                                # this value is computed in generate_movements_quiescence_periods notebook;
# 
#etm.skip_to = 'Mar14_'  # flag to pickup reprocessing at some point; only used if overwrite flag is set to True and need to partially restart
etm.skip_to = None  # 

# 
etm.sessions = 'all'

# 
etm.generate_etm(names,
                 n_sec_window,
                 lockout_window,
                 recompute,
                 pca_etm,
                 pca_explained_var_val,
                 parallel,
                 n_cores,
                 )                


100%|██████████| 71/71 [00:00<00:00, 38659.69it/s]

PROCESSING:  IA1
   recording:  IA1pm_Feb1_30Hz
No video available:  IA1pm_Feb1_30Hz
fname-locs:  /media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_whole_stack_trial_ROItimeCourses_15sec_locs_selected.txt
    computing DFF
missing aligned images - skipping session IA1pm_Feb1_30Hz
data_stm is empty (could not compute stm, skipping :  IA1pm_Feb1_30Hz  )


In [26]:
import pickle
fname = '/media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca.pkl'

file = open(fname,'rb')
object_file = pickle.load(file)
file.close()

print(dir(object_file))
print (np.vstack(object_file.explained_variance_ratio_[:20]))


['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_n_features', '_fit', '_fit_full', '_fit_svd_solver', '_fit_truncated', '_get_param_names', '_get_tags', '_more_tags', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', 'components_', 'copy', 'explained_variance_', 'explained_variance_ratio_', 'fit', 'fit_transform', 'get_covariance', 'get_params', 'get_precision', 'inverse_transform', 'iterated_power', 'mean_', 'n_components', 'n_components_', 'n_features_', 'n_features_in_', 'n_samples_', 'noise_variance_', 'random_state', 'score', 'score_samples', 'set_params', 'singular_values_', 'svd_solver', 't

In [10]:
clrs = ['black','blue','red','green','cyan','magenta','yellow','pink','brown','orange']
d1 = np.load('/media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca_0.95.npy')
print (d1.shape)
for k in range(10):
    plt.plot(d1[:,k]-d1[:,k].mean(0),c=clrs[k])
plt.show()

(1000, 20)


In [28]:
d1 = np.load('/media/cat/1TB/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca_0.95_spatial.npy')
print (d1.shape)
for k in range(20):
    ax=plt.subplot(2,10,k+1)
    plt.imshow(d1[k].reshape(128,128))
    plt.title(str(round(object_file.explained_variance_ratio_[k],4)))
plt.show()

(20, 16384)


In [12]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz_5secNoMove_movements.npz',allow_pickle=True)
labels=data['labels']
print (labels)

['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']


In [5]:
data = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Feb3_30Hz/IJ1pm_Feb3_30Hz_5secNoMove_movements.npz',allow_pickle=True)
features = data['feature_movements']
print (len(features))

7


In [ ]:
OLDER CODE BELOW

In [11]:
#############################################
############## VISUALIZE RESULTS ############
#############################################
    
# LEVER PULL
vis = Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 10
vis.lockout_window = 10
vis.code = 'code_04'
vis.lockout = False

vis.animal_id = "IA1"
vis.session_id = "Feb23_"
title = vis.animal_id + "  "+vis.session_id

#############################################
############## DECISION TYPE ################
#############################################
# select animal and session

# load pca 
fig =plt.figure()

# ax=plt.subplot(2,3,1)
# vis.pca_flag = False
# vis.lockout = True
# vis.pca_var = 0.99
# vis.plot_decision_choice('black',str(vis.pca_var),title, ax)

# vis.lockout = True
# vis.plot_decision_choice('blue',str(vis.pca_var)+" lockout",title, ax)

# 
# vis.lockout = True
# vis.plot_decision_choice('blue',str(vis.pca_var) + " lockout",title, ax)


# 
vars_ = [0.95]
for k in range(len(vars_)):
    #ax=plt.subplot(2,3,k+2)
    ax=plt.subplot(1,1,k+1)
    vis.pca_flag = True
    vis.lockout = False
    vis.pca_var = vars_[k]
    vis.plot_decision_choice('black',str(vis.pca_var),title, ax)

    # 
    vis.pca_flag = True
    vis.lockout = True
    vis.plot_decision_choice('blue',str(vis.pca_var)+" lockout",title, ax)

# vis.pca_flag = True
# vis.pca_var = 0.99
# ax=plt.subplot(2,3,2)
# vis.plot_decision_choice('blue','xxpcs_0.99Variance',title, ax)


# vis.pca_flag = True
# vis.pca_var = 0.95
# ax=plt.subplot(2,3,3)
# vis.plot_decision_choice('red','8pcs_0.95Variance',title, ax)

# vis.pca_flag = True
# vis.pca_var = 0.90
# ax=plt.subplot(2,3,4)
# vis.plot_decision_choice('green','4pcs_0.90Variance',title, ax)


# vis.pca_var = 0.50
# ax=plt.subplot(2,3,5)
# vis.plot_decision_choice('magenta','2pcs_0.50Variance',title, ax)


# fname = vis.main_dir+'/'+animal_id+'/SVM_scores_'+animal_id+'_lockout_'+str(session)+'.npy'
# vis.load_data(fname)
# vis.plot_decision_choice('blue','lockout-denoised')


Self session: IA1pm_Feb23_30Hz
Self session: IA1pm_Feb23_30Hz


In [7]:
import pickle as pk

file = open('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_code_04_lockout_10sec_trial_ROItimeCourses_10sec_pca.pkl', 'rb')

# dump information to that file
pca = pk.load(file)

print (pca.components_.shape)


(1500, 16384)


In [9]:
data_stm = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_stm.npy')
print (data_stm.shape)

(62, 601, 128, 128)


In [10]:
# 
X = data_stm.reshape(data_stm.shape[0]*data_stm.shape[1],
                     data_stm.shape[2]*data_stm.shape[3])

mu= np.mean(X, axis=0)


NameError: name 'nComp' is not defined

In [11]:
nComp = 9
Xnew = np.dot(pca.transform(X)[:,:nComp],
             pca.components_[:nComp,:])

Xnew+=mu

data_stm_denoised_reshaped = Xnew.reshape(data_stm.shape[0], data_stm.shape[1],
                                          data_stm.shape[2], data_stm.shape[3])


In [17]:
time_filters = pca.transform(X)[:,:nComp]
pca_time_filters_only = time_filters.reshape(data_stm.shape[0], data_stm.shape[1],-1).transpose(0,2,1)
print (pca_time_filters_only.shape)


(62, 9, 601)


(62, 601, 9)


In [21]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb23_30Hz/IA1pm_Feb23_30Hz_code_04_trial_ROItimeCourses_10sec_pca_0.5.npy')
print (data.shape)

(82, 2, 601)


In [76]:
# -*- coding: utf-8 -*-
"""Cat Data.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1r-NABuBJOITVlW03VvhsQivPNzmNp4yp
"""

import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import svm
import pandas as pd
from sklearn.metrics import confusion_matrix  
import random
from scipy import stats
from matplotlib.pyplot import MultipleLocator
import os


jobid = os.getenv('SLURM_ARRAY_TASK_ID')
name='IA1_lockout'

data_leverpress = np.load('/media/cat/4TBSSD/yuki/temp/IA1/data_04.npy',allow_pickle=True)[0]
data_random = np.load('/media/cat/4TBSSD/yuki/temp/IA1/data_04_random.npy',allow_pickle=True)[0]

print (data_leverpress.shape)


(62, 35, 601)


In [77]:


#classification of time (10-class)
X_l=data_leverpress #  [n_trials, n_areas, n_time_steps]
X_l=X_l.transpose((0,2,1))


X_R_l= X_l.reshape(-1,X_l.shape[1]*X_l.shape[2])
normal_X_l = preprocessing.normalize(X_R_l)
n_X_l=normal_X_l.reshape(X_l.shape[0],X_l.shape[1],X_l.shape[2])
X_l=n_X_l

X_l=X_l[:,:300,:]
X_k=X_l[:,0:30,:]
X_k=X_k.reshape(X_k.shape[0],X_k.shape[1]*X_k.shape[2])
for i in range(30,271,30):
    X_t = X_l[:,i:i+30,:]
    X_t=X_t.reshape(X_t.shape[0],X_t.shape[1]*X_t.shape[2])
    X_k=np.concatenate((X_k,X_t),axis=0)

y_ct=np.zeros(data_leverpress.shape[0])
for i in range(1,10):
    Z_ct=i*np.ones(data_leverpress.shape[0])
    y_ct=np.concatenate((y_ct,Z_ct))

# 
X_tSVM=X_k # 10s * 30frams
y_tSVM=y_ct # labels

#10-fold confusion matrix
clf = svm.SVC() # Non-linear classifier
ten_svm=[]
conf_matrix_ten=[]
kf = KFold(n_splits=10,random_state=None, shuffle=True) 
for train_index, test_index in kf.split(X_tSVM):
    X_train_k, X_test_k = X_tSVM[train_index], X_tSVM[test_index] 
    y_train_k, y_test_k = y_tSVM[train_index], y_tSVM[test_index]
    clf.fit(X_train_k, y_train_k)
    score=clf.score(X_test_k, y_test_k)
    y_predicted=clf.predict(X_test_k)
    cm=confusion_matrix(y_test_k,y_predicted)
    confusion_m=cm.T # make each row be the prediction
    conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
    conf_matrix_norm = np.nan_to_num(conf_matrix_norm)
    ten_svm.append(score)
    conf_matrix_ten.append(conf_matrix_norm)
np.save('conf_10_'+str(name)+'_'+str(jobid)+'.npy',conf_matrix_ten)
sc=np.mean(conf_matrix_ten,axis=0) # mean of confusion matrix
np.save('conf_avg_'+str(name)+'_'+str(jobid)+'.npy',sc)

<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np.newaxis] #calculate the precision
<ipython-input-77-0c90c545bbff>:41: RuntimeWarning: invalid value encountered in true_divide
  conf_matrix_norm = confusion_m.astype('float') / confusion_m.sum(axis=1)[:,np

In [78]:

#Next is the plotting.

confusion_10=sc.diagonal()
np.save('diagonal_'+str(name)+'_'+str(jobid)+'.npy',confusion_10)
confusion_s=conf_matrix_ten

confusion_d=[]
for i in range(0,10):
    confusion_d.append(confusion_s[i].diagonal())
x_std=np.std(confusion_d,axis=0)/(10**0.5)
x_s=x_std


fig = plt.figure(figsize=(21.5,20), dpi=64, facecolor='white')
plt.xticks(fontsize=60)
plt.yticks(fontsize=60)
axes = plt.subplot(111)
axes.tick_params(axis ='both', which ='both', length = 10,width=4,pad=20)

bwith = 3 
ax = plt.gca()
ax.spines['bottom'].set_linewidth(bwith)
ax.spines['left'].set_linewidth(bwith)
ax.spines['top'].set_linewidth(bwith)
ax.spines['right'].set_linewidth(bwith)
l=15

Y=np.linspace(0,1,12)
X=np.ones(Y.size)
x=np.arange(-9.5,0.5,1)
plt.text(0.1, 0.2, 'Lever Pull', fontdict={'size': 70}, rotation=90)
plt.text(-12.95, 1/10, 'Chance', fontdict={'size': 60}, rotation=0)
plt.axhline(y=1/10, color='black', linestyle='--',linewidth=6)

plt.errorbar(x,confusion_10,x_s,c='red',alpha=0.6,marker='s', mec='red', ms=3, mew=3,label='SVM',linewidth=l,elinewidth=12)

plt.plot( (0+0)*X, Y, color='black',ls='--',linewidth=6)

plt.title('Decoding decision time',fontdict={'size': 110},pad=60)
plt.xlabel('Time before Lever Pull (s)',fontdict={'size': 110},labelpad=60)
plt.ylabel('Decoding Accuracy',fontdict={'size': 110},labelpad=180)
plt.ylim(ymax = 1.09)
ax=plt.gca()
ax.xaxis.set_major_locator(MultipleLocator(2))
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))

plt.xlim(xmin = -10.5)
plt.xlim(xmax = 0.9)

for i in range(0,10):
    a=confusion_10[i]
    b=x_s[i]
    n=10
    s=b
    t=(a-(1/10))/s
    df=n-1
    p = (1 - stats.t.cdf(t,df=df))
    
    if (p > 0.01 and p <= 0.05):
        plt.plot(-9.5+i, confusion_10[i]+x_std[i]+0.06, '*',c='black', ms=20)
        
    if (p > 0.00001 and p <= 0.01):
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.06,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.08,'*',c='black', ms=20)
    if  p <= 0.00001:
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.06,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.08,'*',c='black', ms=20)
        plt.plot(-9.5+i,confusion_10[i]+x_std[i]+0.10,'*',c='black', ms=20)

plt.show()
#plt.savefig('./time_svm_'+str(jobid)+'.jpg')
#plt.close()

<ipython-input-78-40a74f02ec5b>:57: RuntimeWarning: divide by zero encountered in double_scalars
  t=(a-(1/10))/s


In [10]:
data= np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar3_30Hz/IJ2pm_Mar3_30Hz_code_04_lockout_10sec_random_ROItimeCourses_10sec_pca_0.95_spatial.npy')
print (data.shape)

(7, 16384)
